# Deep LDA Covariance Summary

Build a table of mean final accuracies from `deep_lda_covariances.json`.


In [1]:
import json
from pathlib import Path

import pandas as pd

json_path = Path('deep_lda_covariances.json')
with json_path.open('r', encoding='utf-8') as f:
    data = json.load(f)


In [2]:
results = data['results']
rows = []
for dataset_name, covs in results.items():
    for cov_type, runs in covs.items():
        final_test = [r['final_test_acc'] for r in runs]
        mean_test = sum(final_test) / len(final_test)
        variance = sum((x - mean_test) ** 2 for x in final_test) / len(final_test)
        std_test = variance ** 0.5
        rows.append({
            'dataset': dataset_name,
            'covariance': cov_type,
            'mean_test': mean_test,
            'std_test': std_test,
        })

df = pd.DataFrame(rows)
cov_order = ['spherical', 'diag', 'full']
dataset_order = ['FashionMNIST', 'CIFAR10', 'CIFAR100']
df['covariance'] = pd.Categorical(df['covariance'], categories=cov_order, ordered=True)
df['dataset'] = pd.Categorical(df['dataset'], categories=dataset_order, ordered=True)
df['summary'] = df.apply(lambda r: f"{r['mean_test']*100:.2f} \u00b1 {r['std_test']*100:.2f}", axis=1)
table = df.pivot(index='covariance', columns='dataset', values='summary')
table = table.reindex(index=cov_order, columns=dataset_order)
table


dataset,FashionMNIST,CIFAR10,CIFAR100
covariance,,,
spherical,93.38 ± 0.06,90.49 ± 0.26,65.20 ± 0.22
diag,93.32 ± 0.07,90.31 ± 0.26,65.02 ± 0.09
full,93.31 ± 0.11,89.91 ± 0.31,59.64 ± 0.71
